# DSBA 22/23 HSE & University of London

# Practical assignment 1. DL in classification.

## General info
Release data: 26.09.2022

Soft deadline: 10.10.2022 23:59 MSK

Hard deadline: 13.10.2021 23:59 MSK

In this task, you are to build a NN for a binary classification task. We suggest using Google Colab for access to GPU. Competition invite link: https://www.kaggle.com/t/1917e22edb71437ca24d790ab1d57695

## Evaluation and fines

Each section has a defined "value" (in brackets near the section). Maximum grade for the task - 10 points, other points can be assigned to your tests.

**Your notebook with the best solution must be reproducible should be sent to the dropbox!** If the assessor cannot reproduce your results, you may be assigned score = 0, so make all your computations fixed!

**You can only use neural networks / linear / nearest neighbors models for this task - tree-based models are forbidden!**

All the parts must be done independently.

After the hard deadline is passed, the hometask is not accepted. If you send the hometask after the soft deadline, you will be excluded from competition among your mates and the homework will only be scored by the "Beating the baseline" part.

Feel free to ask questions both the teacher and your mates, but __do not copy the code or do it together__. "Similar" solutions are considered a plagiarism and all the involved students (the ones who gave & the ones who did) cannot get more than 0.01 points for the task. If you found a solution in some open source, you __must__ reference it in a special block at the end of your work (to exclude the suspicions in plagiarism).


## Format of handing over

The tasks are sent to the dropbox: https://www.dropbox.com/request/Y6TJouxNbm3r0RgcBL35. Don't forget to attach your name, surname & your group.


## 1. Model training

**Important!** Public Leaderboard contains only 33% of the test data. Your points will be measured wrt to the whole test set, therefore your position on the LB after the end of the competition may change.

* test_accuracy > weak baseline (public LB): 3 points

* test_accuracy > medium baseline (public LB): + 3 points

* test_accuracy > strong baseline (public LB): + 2 points

* You are among 25% most successful students (private LB): + 2 point

* You are among top-3 most successful students (private LB): + 1 point

* You are among top-2 most successful students (private LB): + 1 point

* You are among top-1 most successful students (private LB): + 1 point

In [1]:
# Your code here ╰( ͡° ͜ʖ ͡° )つ──☆*:

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

In [4]:
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [5]:
from tensorflow import keras
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from keras import optimizers

# **Preprocessing data**

In [6]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
target_df = pd.read_csv('train_target.csv')
train_expected_target1 = pd.read_csv('train_expected_target_agent_1.csv')
train_expected_target2 = pd.read_csv('train_expected_target_agent_2.csv')
train_target_agent_1 = pd.read_csv('train_target_agent_1.csv')
train_target_agent_2 = pd.read_csv('train_target_agent_2.csv')

In [7]:
train_target_agent_1 = train_target_agent_1.rename(columns={"0": "expected_target1"})
train_target_agent_2 = train_target_agent_2.rename(columns={"0": "expected_target2"})

In [8]:
train_df = pd.concat([train_df, train_target_agent_1, train_target_agent_2], axis=1)

In [9]:
train_df.head()

,agent_1_feat_Possession%,agent_1_feat_Pass%,agent_1_feat_AerialsWon,agent_1_feat_Rating,agent_1_feat_XGrealiz,agent_1_feat_XGArealiz,agent_1_feat_PPDA,agent_1_feat_OPPDA,agent_1_feat_DC,agent_1_feat_ODC,...,agent_2_feattotal_xg_1,agent_2_feattotal_xg_mean_3,agent_2_feattotal_xg_mean,agent_2_featboth_scored_3,agent_2_featboth_scored_2,agent_2_featboth_scored_1,agent_2_featboth_scored_mean_3,agent_2_featboth_scored_mean,expected_target1,expected_target2
0,58.8,85.1,15.8,6.99,1.143700,0.928715,7.13,14.16,267.0,194.0,...,2.739439,2.739439,NaN,0.473684,0.473684,0.473684,0.473684,NaN,1,2
1,44.8,71.1,23.4,6.84,0.954159,0.975350,9.99,7.66,191.0,287.0,...,2.336756,2.336756,NaN,0.578947,0.578947,0.578947,0.578947,NaN,2,2
2,46.3,70.8,21.7,6.77,0.918434,1.118603,9.56,7.34,179.0,298.0,...,2.120322,2.120322,NaN,0.368421,0.368421,0.368421,0.368421,NaN,0,1
3,50.2,77.5,24.4,6.87,1.037613,0.956836,9.60,9.53,195.0,239.0,...,2.216415,2.216415,NaN,0.210526,0.210526,0.210526,0.210526,NaN,0,1
4,44.9,75.0,17.2,6.77,0.983691,0.948837,12.24,8.76,161.0,283.0,...,2.604025,2.604025,NaN,0.421053,0.421053,0.421053,0.421053,NaN,2,2


In [10]:
test_df.head()

,agent_1_feat_Possession%,agent_1_feat_Pass%,agent_1_feat_AerialsWon,agent_1_feat_Rating,agent_1_feat_XGrealiz,agent_1_feat_XGArealiz,agent_1_feat_PPDA,agent_1_feat_OPPDA,agent_1_feat_DC,agent_1_feat_ODC,...,agent_2_feattotal_xg_3,agent_2_feattotal_xg_2,agent_2_feattotal_xg_1,agent_2_feattotal_xg_mean_3,agent_2_feattotal_xg_mean,agent_2_featboth_scored_3,agent_2_featboth_scored_2,agent_2_featboth_scored_1,agent_2_featboth_scored_mean_3,agent_2_featboth_scored_mean
0,58.6,87.0,15.2,6.83,0.844742,1.165049,9.19,16.50,337.0,179.0,...,2.661870,1.893116,4.241360,2.932115,2.690442,1.0,0.0,1.0,0.666667,0.333333
1,50.7,81.3,14.2,6.65,0.743218,1.152593,10.31,13.63,311.0,208.0,...,3.550724,2.373700,4.197010,3.373811,3.075302,0.0,1.0,1.0,0.666667,0.625000
2,47.3,81.4,17.7,6.73,0.954509,0.956938,14.21,11.82,207.0,270.0,...,2.693652,2.042668,0.966665,1.900995,3.007033,0.0,1.0,1.0,0.666667,0.555556
3,54.5,84.8,14.5,6.85,1.155612,1.049618,10.95,12.46,339.0,186.0,...,3.938100,1.466409,0.922046,2.108852,2.643923,1.0,0.0,0.0,0.333333,0.444444
4,51.3,81.8,16.4,6.81,1.199718,0.856327,11.27,11.52,193.0,293.0,...,3.358338,2.138405,1.872476,2.456406,3.113815,0.0,0.0,0.0,0.000000,0.555556


In [11]:
train_df.shape

(2470, 236)

In [12]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2470 entries, 0 to 2469
Columns: 236 entries, agent_1_feat_Possession% to expected_target2
dtypes: float64(212), int64(24)
memory usage: 4.4 MB


In [13]:
target_df.drop('id', axis = 1, inplace = True)

In [14]:
train_df = pd.concat([target_df, train_df], axis = 1)

In [15]:
train_df

,category,agent_1_feat_Possession%,agent_1_feat_Pass%,agent_1_feat_AerialsWon,agent_1_feat_Rating,agent_1_feat_XGrealiz,agent_1_feat_XGArealiz,agent_1_feat_PPDA,agent_1_feat_OPPDA,agent_1_feat_DC,...,agent_2_feattotal_xg_1,agent_2_feattotal_xg_mean_3,agent_2_feattotal_xg_mean,agent_2_featboth_scored_3,agent_2_featboth_scored_2,agent_2_featboth_scored_1,agent_2_featboth_scored_mean_3,agent_2_featboth_scored_mean,expected_target1,expected_target2
0,1,58.8,85.1,15.8,6.99,1.143700,0.928715,7.13,14.16,267.0,...,2.739439,2.739439,NaN,0.473684,0.473684,0.473684,0.473684,NaN,1,2
1,1,44.8,71.1,23.4,6.84,0.954159,0.975350,9.99,7.66,191.0,...,2.336756,2.336756,NaN,0.578947,0.578947,0.578947,0.578947,NaN,2,2
2,0,46.3,70.8,21.7,6.77,0.918434,1.118603,9.56,7.34,179.0,...,2.120322,2.120322,NaN,0.368421,0.368421,0.368421,0.368421,NaN,0,1
3,0,50.2,77.5,24.4,6.87,1.037613,0.956836,9.60,9.53,195.0,...,2.216415,2.216415,NaN,0.210526,0.210526,0.210526,0.210526,NaN,0,1
4,1,44.9,75.0,17.2,6.77,0.983691,0.948837,12.24,8.76,161.0,...,2.604025,2.604025,NaN,0.421053,0.421053,0.421053,0.421053,NaN,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2465,1,41.6,76.0,17.1,6.62,1.046406,1.032989,18.00,8.27,138.0,...,3.684860,4.024907,3.872622,1.000000,0.000000,0.000000,0.333333,0.444444,1,2
2466,1,42.9,76.1,18.3,6.61,1.161802,1.066236,16.14,7.60,201.0,...,1.568175,2.000313,2.572016,0.000000,0.000000,0.000000,0.000000,0.444444,2,3
2467,0,41.0,72.2,19.1,6.51,1.000858,1.026472,15.99,7.99,164.0,...,3.871643,2.496854,2.555157,0.000000,0.000000,1.000000,0.333333,0.500000,0,5
2468,1,51.4,79.3,14.1,6.62,1.037986,1.161401,9.73,10.47,222.0,...,4.904164,2.977092,2.495116,1.000000,0.000000,0.000000,0.333333,0.222222,1,3


## Delete outliers

In [16]:
train_expected_target1 = train_expected_target1.rename(columns={"0": "train_expected_target1"})
train_expected_target2 = train_expected_target2.rename(columns={"0": "train_expected_target2"})
train_df = pd.concat([train_expected_target1, train_df], axis = 1)
train_df = pd.concat([train_expected_target2, train_df], axis = 1)
train_df.head()

,train_expected_target2,train_expected_target1,category,agent_1_feat_Possession%,agent_1_feat_Pass%,agent_1_feat_AerialsWon,agent_1_feat_Rating,agent_1_feat_XGrealiz,agent_1_feat_XGArealiz,agent_1_feat_PPDA,...,agent_2_feattotal_xg_1,agent_2_feattotal_xg_mean_3,agent_2_feattotal_xg_mean,agent_2_featboth_scored_3,agent_2_featboth_scored_2,agent_2_featboth_scored_1,agent_2_featboth_scored_mean_3,agent_2_featboth_scored_mean,expected_target1,expected_target2
0,0.278076,1.166350,1,58.8,85.1,15.8,6.99,1.143700,0.928715,7.13,...,2.739439,2.739439,NaN,0.473684,0.473684,0.473684,0.473684,NaN,1,2
1,0.613273,1.278300,1,44.8,71.1,23.4,6.84,0.954159,0.975350,9.99,...,2.336756,2.336756,NaN,0.578947,0.578947,0.578947,0.578947,NaN,2,2
2,1.117570,1.900670,0,46.3,70.8,21.7,6.77,0.918434,1.118603,9.56,...,2.120322,2.120322,NaN,0.368421,0.368421,0.368421,0.368421,NaN,0,1
3,0.909774,0.423368,0,50.2,77.5,24.4,6.87,1.037613,0.956836,9.60,...,2.216415,2.216415,NaN,0.210526,0.210526,0.210526,0.210526,NaN,0,1
4,0.991901,1.683430,1,44.9,75.0,17.2,6.77,0.983691,0.948837,12.24,...,2.604025,2.604025,NaN,0.421053,0.421053,0.421053,0.421053,NaN,2,2


In [17]:
print('Rows before deleting: ', train_df.shape[0])
train_df = train_df.drop(train_df[(train_df.train_expected_target2 > 1) &
                                  (train_df.train_expected_target1 > 1) &
                                  (train_df.category == 0)].index)
train_df.drop(['train_expected_target1', 'train_expected_target2'], axis = 1, inplace = True)
print('Rows after deleting: ', train_df.shape[0])

Rows before deleting:  2470
Rows after deleting:  2295


## Work with missing variables

In [18]:
print('Rows before deleting: ', train_df.shape[0])
train_df = train_df.dropna()  
print('Rows after deleting: ', train_df.shape[0])

Rows before deleting:  2295
Rows after deleting:  2163


# Split train and test dataset on two for each agent

In [19]:
test_df

,agent_1_feat_Possession%,agent_1_feat_Pass%,agent_1_feat_AerialsWon,agent_1_feat_Rating,agent_1_feat_XGrealiz,agent_1_feat_XGArealiz,agent_1_feat_PPDA,agent_1_feat_OPPDA,agent_1_feat_DC,agent_1_feat_ODC,...,agent_2_feattotal_xg_3,agent_2_feattotal_xg_2,agent_2_feattotal_xg_1,agent_2_feattotal_xg_mean_3,agent_2_feattotal_xg_mean,agent_2_featboth_scored_3,agent_2_featboth_scored_2,agent_2_featboth_scored_1,agent_2_featboth_scored_mean_3,agent_2_featboth_scored_mean
0,58.6,87.0,15.2,6.83,0.844742,1.165049,9.19,16.50,337.0,179.0,...,2.661870,1.893116,4.241360,2.932115,2.690442,1.0,0.0,1.0,0.666667,0.333333
1,50.7,81.3,14.2,6.65,0.743218,1.152593,10.31,13.63,311.0,208.0,...,3.550724,2.373700,4.197010,3.373811,3.075302,0.0,1.0,1.0,0.666667,0.625000
2,47.3,81.4,17.7,6.73,0.954509,0.956938,14.21,11.82,207.0,270.0,...,2.693652,2.042668,0.966665,1.900995,3.007033,0.0,1.0,1.0,0.666667,0.555556
3,54.5,84.8,14.5,6.85,1.155612,1.049618,10.95,12.46,339.0,186.0,...,3.938100,1.466409,0.922046,2.108852,2.643923,1.0,0.0,0.0,0.333333,0.444444
4,51.3,81.8,16.4,6.81,1.199718,0.856327,11.27,11.52,193.0,293.0,...,3.358338,2.138405,1.872476,2.456406,3.113815,0.0,0.0,0.0,0.000000,0.555556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565,50.8,80.3,16.1,6.72,0.814332,1.292407,13.35,8.32,220.4,304.0,...,2.369897,4.584170,3.757075,3.570381,3.240205,0.0,1.0,0.0,0.333333,0.666667
566,51.8,81.8,14.0,6.69,1.186944,0.769231,14.87,14.67,190.0,266.0,...,3.465800,2.800194,4.670804,3.645599,3.315186,0.0,1.0,0.0,0.333333,0.611111
567,63.1,84.9,15.1,7.05,0.877193,0.257069,9.05,22.23,380.0,197.6,...,1.745647,1.865882,3.505180,2.372236,2.343328,0.0,0.0,1.0,0.333333,0.277778
568,68.2,89.7,12.7,7.11,0.846805,0.401606,7.86,26.84,440.8,129.2,...,3.534070,0.828837,3.122840,2.495249,2.364201,1.0,0.0,1.0,0.666667,0.444444


In [20]:
agent1_test_df = test_df.filter(regex=("agent_1_.*"))
agent2_test_df = test_df.filter(regex=("agent_2_.*"))

In [21]:
train_df = train_df.reset_index(drop=True)
train_df

,category,agent_1_feat_Possession%,agent_1_feat_Pass%,agent_1_feat_AerialsWon,agent_1_feat_Rating,agent_1_feat_XGrealiz,agent_1_feat_XGArealiz,agent_1_feat_PPDA,agent_1_feat_OPPDA,agent_1_feat_DC,...,agent_2_feattotal_xg_1,agent_2_feattotal_xg_mean_3,agent_2_feattotal_xg_mean,agent_2_featboth_scored_3,agent_2_featboth_scored_2,agent_2_featboth_scored_1,agent_2_featboth_scored_mean_3,agent_2_featboth_scored_mean,expected_target1,expected_target2
0,0,44.0,70.3,25.1,6.79,0.711201,0.915529,10.74,9.43,218.0,...,1.608046,2.112304,1.608046,0.578947,0.578947,1.0,0.719298,1.000000,0,0
1,0,57.0,84.6,15.9,7.07,1.094698,0.938272,7.57,13.92,575.0,...,2.479335,2.214160,2.479335,0.526316,0.526316,1.0,0.684211,1.000000,0,1
2,1,48.1,76.9,17.7,6.74,0.994530,1.235052,9.77,8.24,175.0,...,1.712261,2.183093,1.712261,0.526316,0.526316,1.0,0.684211,1.000000,3,3
3,0,46.3,70.8,21.7,6.77,0.918434,1.118603,9.56,7.34,179.0,...,2.675331,2.627794,2.675331,0.421053,0.421053,1.0,0.614035,1.000000,1,0
4,0,50.7,82.1,14.4,6.86,1.124694,0.875939,11.79,10.66,156.0,...,1.331644,2.260683,1.331644,0.368421,0.368421,0.0,0.245614,0.000000,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2158,1,41.6,76.0,17.1,6.62,1.046406,1.032989,18.00,8.27,138.0,...,3.684860,4.024907,3.872622,1.000000,0.000000,0.0,0.333333,0.444444,1,2
2159,1,42.9,76.1,18.3,6.61,1.161802,1.066236,16.14,7.60,201.0,...,1.568175,2.000313,2.572016,0.000000,0.000000,0.0,0.000000,0.444444,2,3
2160,0,41.0,72.2,19.1,6.51,1.000858,1.026472,15.99,7.99,164.0,...,3.871643,2.496854,2.555157,0.000000,0.000000,1.0,0.333333,0.500000,0,5
2161,1,51.4,79.3,14.1,6.62,1.037986,1.161401,9.73,10.47,222.0,...,4.904164,2.977092,2.495116,1.000000,0.000000,0.0,0.333333,0.222222,1,3


In [22]:
Y = train_df['category']

In [23]:
agent1_train_df = train_df.filter(regex=("agent_1_.*"))
agent2_train_df = train_df.filter(regex=("agent_2_.*"))
agent1_train_df = pd.concat([agent1_train_df, train_df['expected_target1']], axis=1)
agent2_train_df = pd.concat([agent2_train_df, train_df['expected_target2']], axis=1)

# Corr

In [24]:
agent1_train_df

,agent_1_feat_Possession%,agent_1_feat_Pass%,agent_1_feat_AerialsWon,agent_1_feat_Rating,agent_1_feat_XGrealiz,agent_1_feat_XGArealiz,agent_1_feat_PPDA,agent_1_feat_OPPDA,agent_1_feat_DC,agent_1_feat_ODC,...,agent_1_feat_total_xg_2.1,agent_1_feat_total_xg_1.1,agent_1_feat_total_xg_mean_3.1,agent_1_feat_total_xg_mean.1,agent_1_feat_both_scored_3.1,agent_1_feat_both_scored_2.1,agent_1_feat_both_scored_1.1,agent_1_feat_both_scored_mean_3.1,agent_1_feat_both_scored_mean.1,expected_target1
0,44.0,70.3,25.1,6.79,0.711201,0.915529,10.74,9.43,218.0,258.0,...,2.389219,2.501101,2.426513,2.501101,0.421053,0.421053,1.0,0.614035,1.000000,0
1,57.0,84.6,15.9,7.07,1.094698,0.938272,7.57,13.92,575.0,144.0,...,3.181885,2.215639,2.859803,2.215639,0.421053,0.421053,1.0,0.614035,1.000000,0
2,48.1,76.9,17.7,6.74,0.994530,1.235052,9.77,8.24,175.0,239.0,...,2.461948,2.446113,2.456670,2.446113,0.578947,0.578947,0.0,0.385965,0.000000,3
3,46.3,70.8,21.7,6.77,0.918434,1.118603,9.56,7.34,179.0,298.0,...,2.946534,3.018240,2.970436,3.018240,0.526316,0.526316,0.0,0.350877,0.000000,1
4,50.7,82.1,14.4,6.86,1.124694,0.875939,11.79,10.66,156.0,310.0,...,2.357624,1.757546,2.157598,1.757546,0.526316,0.526316,0.0,0.350877,0.000000,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2158,41.6,76.0,17.1,6.62,1.046406,1.032989,18.00,8.27,138.0,322.0,...,2.138405,0.966665,1.826257,2.318888,1.000000,0.000000,1.0,0.666667,0.666667,1
2159,42.9,76.1,18.3,6.61,1.161802,1.066236,16.14,7.60,201.0,272.0,...,2.042668,0.922046,2.107684,2.307469,0.000000,1.000000,0.0,0.333333,0.555556,2
2160,41.0,72.2,19.1,6.51,1.000858,1.026472,15.99,7.99,164.0,343.0,...,1.977760,4.904164,3.180136,3.104836,0.000000,0.000000,0.0,0.000000,0.222222,0
2161,51.4,79.3,14.1,6.62,1.037986,1.161401,9.73,10.47,222.0,275.0,...,1.743456,1.872476,1.786298,2.076611,0.000000,0.000000,0.0,0.000000,0.222222,1


In [25]:
corr_matrix1 = agent1_train_df.corr()
print(corr_matrix1["expected_target1"].sort_values(ascending=False))

expected_target1           1.000000
agent_1_feat_ScoredAv      0.385431
agent_1_feat_XgAv          0.360820
agent_1_feat_DC            0.341588
agent_1_feat_pl_median     0.328869
                             ...   
agent_1_feat_AerialsWon   -0.195486
agent_1_feat_PPDA         -0.213783
agent_1_feat_MissedAv     -0.260551
agent_1_feat_XgaAv        -0.267009
agent_1_feat_ODC          -0.290223
Name: expected_target1, Length: 118, dtype: float64


## Split dataset on train and test

In [26]:
X1 = agent1_train_df.drop('expected_target1', axis=1)
Y1 = agent1_train_df['expected_target1']
X_train1, X_test1, y_train1, y_test1 = (X1.iloc[0:int(len(X1)*0.8)], 
                                    X1.iloc[int(len(X1)*0.8):len(X1)-1], 
                                    Y1.iloc[0:int(len(Y1)*0.8)], 
                                    Y1.iloc[int(len(Y1)*0.8):len(Y1)-1])

In [27]:
X_train1.shape, X_test1.shape, y_train1.shape, y_test1.shape

((1730, 117), (432, 117), (1730,), (432,))

In [28]:
X2 = agent2_train_df.drop('expected_target2', axis=1)
Y2 = agent2_train_df['expected_target2']
X_train2, X_test2, y_train2, y_test2 = (X2.iloc[0:int(len(X2)*0.8)], 
                                    X2.iloc[int(len(X2)*0.8):len(X2)-1], 
                                    Y2.iloc[0:int(len(Y2)*0.8)], 
                                    Y2.iloc[int(len(Y2)*0.8):len(Y2)-1])

In [29]:
X_train2.shape, X_test2.shape, y_train2.shape, y_test2.shape

((1730, 117), (432, 117), (1730,), (432,))

## Scale data

In [30]:
ss1 = StandardScaler()
X_train1 = ss1.fit_transform(X_train1)
X_test1 = ss1.transform(X_test1)
agent1_test_df = ss1.transform(agent1_test_df)

In [31]:
ss2 = StandardScaler()
X_train2 = ss2.fit_transform(X_train2)
X_test2 = ss2.transform(X_test2)
agent2_test_df = ss1.transform(agent2_test_df)

# **Model**

In [32]:
batch_size = [32, 64, 128, 256]
epochs = [50, 75, 150]
optimizer = ['SGD', 'Adam']
learning_rate = [0.001, 0.0001]
param_opt = dict(batch_size=batch_size, epochs=epochs, learning_rate=learning_rate, optimizer = optimizer)

In [33]:
def create_model(batch_size, epochs, learning_rate, optimizer, num_classes, input_shape):
    model = Sequential()
    model.add(Dense(128, activation="relu", input_shape=(input_shape, ))) # Hidden Layer 1

    model.add(Dense(64, activation="relu")) # Hidden Layer 3
    model.add(Dropout(0.2))

    model.add(Dense(64, activation="relu")) # Hidden Layer 4
    model.add(Dropout(0.2))
    
    model.add(Dense(32, activation="relu")) # Hidden Layer 3
    model.add(Dropout(0.2))
    
    model.add(Dense(num_classes, activation="softmax")) # Outout Layer
    
#     opt = optimizers.Adam(learning_rate=learning_rate)

    model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

    return model

In [34]:
# model_GridSearch = KerasClassifier(build_fn=create_model, verbose=0)
# grid = GridSearchCV(estimator=model_GridSearch, param_grid=param_opt, n_jobs=1, cv=3, verbose = 0)
# grid_result = grid.fit(X_train, y_train)

In [35]:
# print('Best parameters are: ')
# print('batch_size: ' + str(grid_result.best_params_['batch_size']))
# print('epochs: ' + str(grid_result.best_params_['epochs']))
# print('optimizer: ' + str(grid_result.best_params_['optimizer']))
# print('learning_rate: ' + str(grid_result.best_params_['learning_rate']))

In [36]:
# batch_size = grid_result.best_params_['batch_size']
# epochs = grid_result.best_params_['epochs']
# learning_rate = grid_result.best_params_['learning_rate']
# optimizer = grid_result.best_params_['optimizer']

# Model for Agent1

In [37]:
batch_size = 32
epochs = 50
learning_rate = 0.001
optimizer = 'Adam'

In [38]:
y_train1 = pd.get_dummies(y_train1)
y_test1 = pd.get_dummies(y_test1)

In [39]:
dif = len(y_train1.columns) - len(y_test1.columns)
if dif == 0:
    print('finish')
elif dif > 0:
    while dif != 0:
        y_test1[int(y_test1.columns[-1])+1] = 0
        dif -= 1
else:
    while dif != 0:
        y_train1[int(y_train1.columns[-1])+1] = 0
        dif += 1

In [40]:
y_train1

,0,1,2,3,4,5,6,7,8
0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
1725,0,1,0,0,0,0,0,0,0
1726,1,0,0,0,0,0,0,0,0
1727,0,1,0,0,0,0,0,0,0
1728,0,1,0,0,0,0,0,0,0


In [41]:
y_test1

,0,1,2,3,4,5,6,7,8
1730,0,1,0,0,0,0,0,0,0
1731,0,0,1,0,0,0,0,0,0
1732,0,0,1,0,0,0,0,0,0
1733,1,0,0,0,0,0,0,0,0
1734,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
2157,0,0,1,0,0,0,0,0,0
2158,0,1,0,0,0,0,0,0,0
2159,0,0,1,0,0,0,0,0,0
2160,1,0,0,0,0,0,0,0,0


In [42]:
num_classes = agent1_train_df['expected_target1'].nunique()
model1 = create_model(batch_size, epochs, learning_rate, optimizer, num_classes, X_train1.shape[1])

In [43]:
history1 = model1.fit(X_train1, y_train1, batch_size = batch_size, epochs = epochs, shuffle = True)

Epoch 1/50
55/55 [==============================] - 12s 1ms/step - loss: 2.2287 - accuracy: 0.1422
Epoch 2/50
55/55 [==============================] - 0s 997us/step - loss: 1.6515 - accuracy: 0.3065
Epoch 3/50
55/55 [==============================] - 0s 997us/step - loss: 1.6161 - accuracy: 0.3306
Epoch 4/50
55/55 [==============================] - 0s 1ms/step - loss: 1.5936 - accuracy: 0.3114
Epoch 5/50
55/55 [==============================] - 0s 1ms/step - loss: 1.5370 - accuracy: 0.3485
Epoch 6/50
55/55 [==============================] - 0s 1ms/step - loss: 1.5260 - accuracy: 0.3619
Epoch 7/50
55/55 [==============================] - 0s 1ms/step - loss: 1.4978 - accuracy: 0.3602
Epoch 8/50
55/55 [==============================] - 0s 1ms/step - loss: 1.4658 - accuracy: 0.3758
Epoch 9/50
55/55 [==============================] - 0s 1ms/step - loss: 1.4451 - accuracy: 0.3646
Epoch 10/50
55/55 [==============================] - 0s 968us/step - loss: 1.4148 - accuracy: 0.3952
Epoch 11/50


In [44]:
validation_loss, validation_accuracy = model1.evaluate(X_test1, y_test1, batch_size=batch_size)
print("Loss: "+ str(np.round(validation_loss, 3)))
print("Accuracy: "+ str(np.round(validation_accuracy, 3)))

14/14 [==============================] - 0s 690us/step - loss: 4.7702 - accuracy: 0.2685
Loss: 4.77
Accuracy: 0.269


In [45]:
y1_pred = model1.predict(X_test1)
tmp1 = []
for i in y1_pred:
    if i.argmax() != 0:
        tmp1.append(1)
    else:
        tmp1.append(0)

print(tmp1)

[1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

# Model for agent2

In [46]:
batch_size = 32
epochs = 50
learning_rate = 0.001
optimizer = 'Adam'

In [47]:
y_train2 = pd.get_dummies(y_train2)
y_test2 = pd.get_dummies(y_test2)

In [48]:
dif = len(y_train2.columns) - len(y_test2.columns)
if dif == 0:
    print('finish')
elif dif > 0:
    while dif != 0:
        y_test2[int(y_test2.columns[-1])+1] = 0
        dif -= 1
else:
    while dif != 0:
        y_train2[int(y_train2.columns[-1])+1] = 0
        dif += 1

In [49]:
y_train2

,0,1,2,3,4,5,6,7,8
0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
1725,0,0,0,0,0,1,0,0,0
1726,1,0,0,0,0,0,0,0,0
1727,0,0,1,0,0,0,0,0,0
1728,0,0,0,0,1,0,0,0,0


In [50]:
y_test2

,0,1,2,3,4,5,6,7,9
1730,1,0,0,0,0,0,0,0,0
1731,1,0,0,0,0,0,0,0,0
1732,0,1,0,0,0,0,0,0,0
1733,1,0,0,0,0,0,0,0,0
1734,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
2157,1,0,0,0,0,0,0,0,0
2158,0,0,1,0,0,0,0,0,0
2159,0,0,0,1,0,0,0,0,0
2160,0,0,0,0,0,1,0,0,0


In [51]:
num_classes = agent2_train_df['expected_target2'].nunique()
model2 = create_model(batch_size, epochs, learning_rate, optimizer, num_classes, X_train2.shape[1])

In [52]:
history2 = model2.fit(X_train2, y_train2, batch_size = batch_size, epochs = epochs, shuffle = True)

Epoch 1/50
55/55 [==============================] - 1s 2ms/step - loss: 1.8242 - accuracy: 0.2701
Epoch 2/50
55/55 [==============================] - 0s 831us/step - loss: 1.4739 - accuracy: 0.3566
Epoch 3/50
55/55 [==============================] - 0s 856us/step - loss: 1.4254 - accuracy: 0.3699
Epoch 4/50
55/55 [==============================] - 0s 850us/step - loss: 1.4052 - accuracy: 0.3811
Epoch 5/50
55/55 [==============================] - 0s 904us/step - loss: 1.3624 - accuracy: 0.3922
Epoch 6/50
55/55 [==============================] - 0s 831us/step - loss: 1.3280 - accuracy: 0.4394
Epoch 7/50
55/55 [==============================] - 0s 1ms/step - loss: 1.3023 - accuracy: 0.4256
Epoch 8/50
55/55 [==============================] - 0s 1ms/step - loss: 1.2568 - accuracy: 0.4534
Epoch 9/50
55/55 [==============================] - 0s 831us/step - loss: 1.2403 - accuracy: 0.4690
Epoch 10/50
55/55 [==============================] - 0s 842us/step - loss: 1.1994 - accuracy: 0.5026
Epoch

In [53]:
validation_loss, validation_accuracy = model2.evaluate(X_test2, y_test2, batch_size=batch_size)
print("Loss: "+ str(np.round(validation_loss, 3)))
print("Accuracy: "+ str(np.round(validation_accuracy, 3)))

14/14 [==============================] - 0s 614us/step - loss: 4.7522 - accuracy: 0.3171
Loss: 4.752
Accuracy: 0.317


In [54]:
y2_pred = model2.predict(X_test2)
tmp2 = []
for i in y2_pred:
    if i.argmax() != 0:
        tmp2.append(1)
    else:
        tmp2.append(0)

print(tmp2)

[1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 

# Test acc

In [55]:
test_pred = np.logical_and(tmp1, tmp2)
test_pred = [0 if x==False else x for x in test_pred]
test_pred = [1 if x==True else x for x in test_pred]

In [56]:
test_real_value = Y[int(len(Y2)*0.8):len(Y2)-1]
test_real_value

1730    0
1731    0
1732    1
1733    0
1734    0
       ..
2157    0
2158    1
2159    1
2160    0
2161    1
Name: category, Length: 432, dtype: int64

In [57]:
acc = 0
for elem1, elem2 in zip(test_pred, test_real_value):
    if elem1 == elem2:
        acc += 1
print("ACC: ", np.round((acc / len(test_real_value)), 3))

ACC:  0.525


# Make a submission

In [58]:
fin_pred1 = model1.predict(agent1_test_df)
tmp1 = []
for i in fin_pred1:
    if i.argmax() != 0:
        tmp1.append(1)
    else:
        tmp1.append(0)

print(tmp1)

[1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 

In [59]:
fin_pred2 = model2.predict(agent2_test_df)
tmp2 = []
for i in fin_pred2:
    if i.argmax() != 0:
        tmp2.append(1)
    else:
        tmp2.append(0)

print(tmp2)

[1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 

In [60]:
Answer = np.logical_and(tmp1, tmp2)
Answer = [0 if x==False else x for x in Answer]
Answer = [1 if x==True else x for x in Answer]

In [61]:
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission['tmp'] = Answer
sample_submission.drop(['category'], axis = 1, inplace= True)
sample_submission = sample_submission.rename(columns={"tmp": "category"})
sample_submission.to_csv('Answer.csv', index = False)

In [62]:
sample_submission

,id,category
0,0,1
1,1,1
2,2,0
3,3,1
4,4,0
...,...,...
565,565,1
566,566,1
567,567,0
568,568,1
